# Resource Catalog

__Description & purpose__: This Notebook is designed to showcase the initial functionality of the Earth Observation Data Hub. It provides a snapshot of the Hub, the `pyeodh` API client and the various datasets as of December 2024.   

__Author(s)__: Alastair Graham, Dusan Figala, EODH

__Date created__: 2024-09-05

__Date last modified__: 2024-12-11

__Licence__: This notebook is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/).  The code is released using the [BSD-2-Clause](https://www.tldrlegal.com/license/bsd-2-clause-license-freebsd) license.


<span style="font-size:0.75em;">
Copyright (c) , All rights reserved.</span>

<span style="font-size:0.75em;">
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="font-size:0.75em;">
Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

[WIP Icon Attribution]("https://www.flaticon.com/free-icons/work-in-progress")

The first thing to do is ensure that the most recent version of `pyeodh` is installed on your system. It is good practice to run the following cell if you have not installed `pyeodh` or have not used it in a while.

In [ ]:
# Run this cell if pyeodh is not installed, or needs updating
!pip install --upgrade pyeodh

# Exploring the Resource Catalogue

Now we are ready to investigate the Resource Catalogue. First off, we need to import the pyeodh package.

In [1]:
# Import the Python API Client
import pyeodh

Next we need to create an instance of the `Client`, which is our entrypoint to EODH APIs. From there we can start to search the collections held within the platform.

In [2]:
# Connect to the Hub
client = pyeodh.Client().get_catalog_service()

# Print a list of the collections held in the Resource Catalogue (their id and description).
# As the Resource Catalogue fills and development continues, the number of collections and the richness of their descriptions will increase
for collect in client.get_collections():
    print(f"- {collect.id}: {collect.description}")

- ukcp: UKCP
- sentinel2_ard: sentinel 2 ARD
- sentinel1: Sentinel 1
- sentinel-2-l2a: Global Sentinel-2 data from the Multispectral Instrument (MSI) onboard Sentinel-2
- sentinel-2-l1c: Global Sentinel-2 data from the Multispectral Instrument (MSI) onboard Sentinel-2
- sentinel-2-c1-l2a: Sentinel-2 Collection 1 L2A, data from the Multispectral Instrument (MSI) onboard Sentinel-2
- sentinel-1-grd: Sentinel-1 is a pair of Synthetic Aperture Radar (SAR) imaging satellites launched in 2014 and 2016 by the European Space Agency (ESA). Their 6 day revisit cycle and ability to observe through clouds makes this dataset perfect for sea and land monitoring, emergency response due to environmental disasters, and economic applications. This dataset represents the global Sentinel-1 GRD archive, from beginning to the present, converted to cloud-optimized GeoTIFF format.
- naip: The [National Agriculture Imagery Program](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-progr

The attributes of a catalogue are mapped to a series of properties. For instance, in the following cell we print the `id`, `title` and `description` for the supported-datasets catalogue. 

In [3]:
ceda_cat = client.get_catalog("supported-datasets/ceda-stac-catalogue")
print("id: ", ceda_cat.id)
print("title: ", ceda_cat.title)
print("description: ", ceda_cat.description)

id:  ceda-stac-catalogue
title:  stac-fastapi-elasticsearch
description:  stac-fastapi-elasticsearch


The Hub API endpoints are wrapped in methods inside `pyeodh` and are structured into classes, following the same logic as the underlying APIs. This means that, for xample, to fetch a collection item we first need to get the collection from the resource catalogue. The following cell provedes a code example to do this. 

In [4]:
# GET /stac-fastapi/collections/{collection_id}/items/{item_id}
cmip6 = client.get_catalog("supported-datasets/ceda-stac-catalogue").get_collection('cmip6')
cmip6

Some API responses are paginated (e.g. collection items), and you can simply iterate over them.

In [ ]:
# GET /stac-fastapi/collections/cmip6/items
items = cmip6.get_items()

# Warning: this will take a long time for large catalogues such as cmip6
for item in items:
    print(item.id)


# Creating, removing and updating collections

![](../img/wip.png){width=50px} 

**To be added**
* Fix the following cells
* Improve explanations of the code blocks


Attempting to create a collection with id that already exists will result in 409 error code. To see the example in action delete the test collection first by running the cell below.

Delete a colletion

In [ ]:
client.get_collection("test1").delete()

Create new collection example

In [ ]:
test1 = client.create_collection(id="test1", title="Test", description="Test collection")
print(test1.description)

Update a collection

In [ ]:
test1.update(description="Different description")
print(test1.description)

# Interacting with items

Create an item

In [ ]:
testitem1 = test1.create_item(id="test1.testitem1")
print(f"Created {testitem1.id} in collection {testitem1.collection}")

Update an item

In [ ]:
testitem1.update(properties={"foo": "bar"})
print(testitem1.properties)

Delete an item

In [ ]:
testitem1.delete()

Find out more about the Resource Catalog

In [ ]:
print(f"Livecheck: PING-{rc.ping()}")
print("\nAPI conforms to:", *rc.get_conformance(), sep="\n")